## data

In [55]:
import pandas as pd
import os
import openpyxl

In [56]:
os.chdir(r'E:/STU/Lab/(230105)QFD/Practice(Bosch)/code')
os.getcwd()

'E:\\STU\\Lab\\(230105)QFD\\Practice(Bosch)\\code'

In [57]:
cr = pd.read_csv("amazon_WAJ2416WIN.csv")

In [58]:
cr.drop(['Unnamed: 0'],axis=1,inplace=True)

In [59]:
cr.columns = ['Date','Star','title','body']
cr

,Date,Star,title,body
0,2023-02-20,5,Durable Solid excellent,Superlike
1,2023-02-14,5,Great quality,Fantastic
2,2022-12-08,4,Nice Product,Everything Nice. The Filter connection fitting...
3,2023-01-22,5,After 1.5year Review | Excellence at it best,Posting this review after 1.5year of usage.Thi...
4,2023-01-28,5,Nicee product,Washing quality is praise worthy.
...,...,...,...,...
2154,2019-11-05,5,Less noise,Less noise
2155,2019-12-18,5,Easy to use.,Nice product. Better than most popular company...
2156,2020-03-10,5,Good product,It’s been 3 months since we have been using it...
2157,2021-11-03,4,so far so good,Too early to give my considered opinion. Tumbl...


## translate
- 한국어 불용어 처리가 잘 안됨
- 영어로 바꿔보면 어떨까
- 영어사전은 잘 되어 있는 것 같음

- 생각보다 번역이 잘됨
ex) 아쥬 좋아요! -> i like it (오타 같은건 아예 빼버리는 듯)

In [60]:
import googletrans

def translate(df):
    translator = googletrans.Translator()
    
    for i in range(len(df)):    
        try:
            result = translator.translate(df['body'][i],dest='en')
            print(i,result.text)
            df['body'][i] = result.text
        except:
            continue

In [61]:
translate(cr)

In [62]:
cr

,Date,Star,title,body
0,2023-02-20,5,Durable Solid excellent,Superlike
1,2023-02-14,5,Great quality,Fantastic
2,2022-12-08,4,Nice Product,Everything Nice. The Filter connection fitting...
3,2023-01-22,5,After 1.5year Review | Excellence at it best,Posting this review after 1.5year of usage.Thi...
4,2023-01-28,5,Nicee product,Washing quality is praise worthy.
...,...,...,...,...
2154,2019-11-05,5,Less noise,Less noise
2155,2019-12-18,5,Easy to use.,Nice product. Better than most popular company...
2156,2020-03-10,5,Good product,It’s been 3 months since we have been using it...
2157,2021-11-03,4,so far so good,Too early to give my considered opinion. Tumbl...


In [63]:
cr.to_csv("cr_translate_WAJ2416WIN.csv")

## 전처리?

In [64]:
import pickle
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm

cr['clean_text'] = pd.Series(dtype='object')
cr['stopwords_after'] = pd.Series(dtype='object')
cr['pos_tag'] = pd.Series(dtype='object')
cr['NN'] = pd.Series(dtype='object')
cr['lemmatization'] = pd.Series(dtype='object')

stops = stopwords.words('english')
stops.append('machine')
stops.append('product')
stops.append('bosch')

def clean_text(texts): 
    corpus = []
    
    for i in tqdm(range(0, len(texts))):
        
        body = texts[i]
        
        body = re.sub('[^a-zA-Z]', ' ', body) # 특수문자 제거 
        body = body.lower().split() # 대문자를 소문자로 변경, 문장을 단어 단위로 구분
        
        cr['clean_text'][i] = body
        
        no_stops = [word for word in body if not word in stops] # 불용어 제거
        cr['stopwords_after'][i] = no_stops
        
        tokens_pos = nltk.pos_tag(cr['stopwords_after'][i]) # pos tagging (품사 태깅)
        cr['pos_tag'][i] = tokens_pos
        
        NN_words = [] # 명사는 NN을 포함하고 있음을 알 수 있음(명사만 추출)
        for word, pos in tokens_pos:
            if 'NN' in pos:
                NN_words.append(word)
                cr['NN'][i] = NN_words
                
        wlem = nltk.WordNetLemmatizer() # Lemmatization(원형(lemma) 찾기) # nltk에서 제공되는 WordNetLemmatizer을 이용
        lemmatized_words = []
        for word in NN_words:
            new_word = wlem.lemmatize(word)
            lemmatized_words.append(new_word)
            cr['lemmatization'][i] = lemmatized_words
        
        corpus.append(no_stops) 
        
    return corpus

In [65]:
cr.to_csv("cr_beforeclean_WAJ2416WIN.csv")

In [66]:
clean_text(cr['body'])

  0%|                                                                                         | 0/2159 [00:00<?, ?it/s]C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['clean_text'][i] = body
C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['stopwords_after'][i] = no_stops
C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

TypeError: expected string or bytes-like object

In [ ]:
cr

## LDA

In [30]:
cr.to_csv("cr_cleaning_WAJ2416WIN.csv")
cr

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


In [31]:
cr

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


In [ ]:
cr.info()

In [ ]:
cr2 = cr
cr2

In [ ]:
##길이가 2 이하인 문자 제거
cr2['lemmatization'] = cr2['lemmatization'].replace("[^0-9-_.]","")
cr2 = cr2.dropna()
cr2.info()

In [ ]:
import warnings
from gensim.corpora.dictionary import Dictionary
from gensim import corpora

In [ ]:
dictionary = corpora.Dictionary(cr2['lemmatization'])
corpus = [dictionary.doc2bow(text) for text in cr2['lemmatization']]

In [39]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

(0, '0.041*"time" + 0.040*"wash" + 0.025*"program" + 0.023*"spin" + 0.023*"use" + 0.016*"feature" + 0.015*"drum" + 0.014*"door"')
(1, '0.029*"value" + 0.029*"money" + 0.024*"use" + 0.021*"year" + 0.020*"programme" + 0.019*"delivery" + 0.017*"love" + 0.015*"cycle"')
(2, '0.018*"time" + 0.015*"spin" + 0.015*"cycle" + 0.015*"program" + 0.015*"week" + 0.012*"drum" + 0.012*"foot" + 0.010*"use"')
(3, '0.025*"time" + 0.021*"excellent" + 0.020*"delivery" + 0.017*"water" + 0.017*"panel" + 0.013*"problem" + 0.013*"control" + 0.013*"team"')
(4, '0.086*"use" + 0.035*"install" + 0.030*"job" + 0.019*"washer" + 0.018*"day" + 0.016*"amazon" + 0.012*"option" + 0.010*"date"')
(5, '0.043*"quality" + 0.031*"year" + 0.029*"clothes" + 0.022*"wash" + 0.022*"washing" + 0.018*"cycle" + 0.017*"program" + 0.013*"installation"')
(6, '0.068*"cycle" + 0.038*"wash" + 0.033*"spin" + 0.031*"clothes" + 0.023*"delivery" + 0.022*"time" + 0.021*"use" + 0.013*"hour"')


In [40]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [41]:
##머신 단어 제거
cr2['lemmatization'] = cr2['lemmatization'].replace("machine","")
cr2 = cr2.dropna()
cr2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274 entries, 0 to 292
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             274 non-null    object
 1   Star             274 non-null    int64 
 2   title            274 non-null    object
 3   body             274 non-null    object
 4   clean_text       274 non-null    object
 5   stopwords_after  274 non-null    object
 6   pos_tag          274 non-null    object
 7   NN               274 non-null    object
 8   lemmatization    274 non-null    object
dtypes: int64(1), object(8)
memory usage: 21.4+ KB


C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\793555062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr2['lemmatization'] = cr2['lemmatization'].replace("machine","")


- machine 제거

In [42]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

(0, '0.024*"delivery" + 0.021*"installation" + 0.019*"use" + 0.019*"door" + 0.017*"time" + 0.016*"drum" + 0.016*"love" + 0.014*"guy"')
(1, '0.033*"delivery" + 0.027*"use" + 0.026*"wash" + 0.021*"service" + 0.021*"amazon" + 0.019*"program" + 0.017*"cycle" + 0.013*"lot"')
(2, '0.056*"cycle" + 0.029*"wash" + 0.028*"time" + 0.022*"clothes" + 0.019*"year" + 0.015*"programme" + 0.015*"lot" + 0.013*"day"')
(3, '0.047*"wash" + 0.045*"time" + 0.031*"value" + 0.026*"day" + 0.025*"money" + 0.013*"performance" + 0.013*"program" + 0.013*"spin"')
(4, '0.029*"quality" + 0.028*"spin" + 0.026*"year" + 0.025*"clothes" + 0.017*"job" + 0.017*"delivery" + 0.017*"thing" + 0.016*"cycle"')
(5, '0.061*"use" + 0.033*"program" + 0.029*"cycle" + 0.026*"wash" + 0.021*"washer" + 0.021*"programme" + 0.015*"spin" + 0.015*"hour"')
(6, '0.030*"use" + 0.023*"excellent" + 0.019*"clean" + 0.019*"amazon" + 0.014*"plumber" + 0.014*"install" + 0.014*"item" + 0.014*"wife"')


In [67]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


- machine, product, bosch 제거

In [68]:
import gensim

NUM_TOPICS = 11
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

(0, '0.027*"time" + 0.023*"use" + 0.023*"guy" + 0.018*"delivery" + 0.015*"clothes" + 0.014*"amazon" + 0.014*"fault" + 0.014*"pipe"')
(1, '0.060*"wash" + 0.042*"cycle" + 0.041*"value" + 0.033*"money" + 0.027*"quality" + 0.024*"spin" + 0.023*"use" + 0.017*"delivery"')
(2, '0.119*"use" + 0.058*"program" + 0.039*"job" + 0.020*"love" + 0.019*"drum" + 0.019*"time" + 0.016*"clothes" + 0.016*"spin"')
(3, '0.084*"cycle" + 0.036*"delivery" + 0.026*"clothes" + 0.023*"time" + 0.022*"wash" + 0.015*"door" + 0.015*"thing" + 0.015*"service"')
(4, '0.035*"day" + 0.030*"washing" + 0.020*"speed" + 0.020*"time" + 0.018*"panel" + 0.018*"temperature" + 0.017*"year" + 0.013*"control"')
(5, '0.038*"clothes" + 0.032*"spin" + 0.032*"excellent" + 0.021*"item" + 0.021*"feature" + 0.016*"capacity" + 0.011*"time" + 0.011*"team"')
(6, '0.056*"use" + 0.038*"install" + 0.024*"time" + 0.020*"money" + 0.019*"programme" + 0.019*"day" + 0.016*"installation" + 0.016*"nothing"')
(7, '0.028*"installation" + 0.023*"lot" + 0.0

In [69]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
